## Loading Data

In [1]:
import pandas as pd
import os
from src.data_processing import load_dfs, create_combined_df

In [2]:
dfs = load_dfs()

Loaded 2018.csv with 112 rows. Null Entries: 0
Loaded 2019.csv with 154 rows. Null Entries: 0
Loaded 2020.csv with 109 rows. Null Entries: 0
Loaded 2021.csv with 251 rows. Null Entries: 0
Loaded 2022.csv with 213 rows. Null Entries: 0
Loaded 2023.csv with 296 rows. Null Entries: 0
Loaded 2024.csv with 216 rows. Null Entries: 0


In [3]:
df_combined = create_combined_df(dfs)
df_combined.to_csv("data/combined.csv", index=False, encoding="utf-8")
print(f"Combined Null Entries: {df_combined.isnull().sum().sum()}")

Combined Null Entries: 0


## Create categories

In [4]:
from openai import OpenAI
from src.data_classification import generate_categories

In [5]:
api_key = input("Enter your OpenAI API key: ")
client = OpenAI(api_key=api_key)
items = df_combined['Item'].unique()

In [6]:
lines = generate_categories(client, items).split("\n")
for line in lines:
    print(line)

Certainly! Here is a practical set of categories for your purchases:

Category Details:
- **Food & Beverages**: This category includes all purchases related to meals, snacks, and drinks, whether consumed at home or outside.  
  Example items (Only 3): Chicken Chop (AMK HUB), Coke (Cheers), Pizza and Coke (Pezzo)

- **Digital Subscriptions & Services**: This includes recurring digital services and software subscriptions.  
  Example items (Only 3): Netflix Subscription, Spotify Premium Subscription, Adobe Creative Cloud (Student Plan)

- **Books & Literature**: This category covers all book purchases, including novels, educational books, and manga/light novels.  
  Example items (Only 3): Halo: Legacy of Onyx (Book), The Last Wish (Book), Sword Art Online Progressive 5 (Kindle)

- **Gaming**: This includes video games, in-game purchases, and gaming-related subscriptions.  
  Example items (Only 3): $100 Xbox Gift Card, Destiny 2 - 3000 Silver, Borderlands 3 (Digital)

- **Collectibles**

## Update CSVs with new categories

In [6]:
metadata_df = pd.read_csv("metadata/categories.csv", encoding="utf-8")
metadata_df.head()

,Category,Description,Examples
0,Food & Beverages,"Items related to meals, snacks, and drinks — i...","McDonalds, Waffle (NYP), Coke"
1,Books & Literature,"Physical and digital books, including manga, l...","Sword Art Online Progressive (Kindle), The Las..."
2,Gaming,"Video games (digital or physical), in-game pur...","Destiny 2 - 3000 Silver, Xbox Series X, Border..."
3,Digital Subscriptions,Recurring or one-time payments for digital ser...,"Netflix Subscription, Adobe Creative Cloud, Sp..."
4,Entertainment,"Movies (tickets, rentals, DVDs), non-gaming me...","Avengers Tickets, Thor: Ragnarok DVD, Solo: A ..."


In [7]:
categories = []
for index, row in metadata_df.iterrows():
    name = row["Category"]
    description = row["Description"]
    example = row["Examples"]
    
    category_string = f"""**{name}**: {description}
    Example items (Only 3): {example}"""
    categories.append(category_string)
    
for category in categories:
    print(category)
    break

**Food & Beverages**: Items related to meals, snacks, and drinks — including dining out, takeaway, and groceries.
    Example items (Only 3): McDonalds, Waffle (NYP), Coke


In [8]:
from src.data_classification import classify_item, classify_items_in_df
category_string = "\n".join(categories)

In [10]:
print(classify_item(client, "Nintendo Switch", category_string))
print(classify_item(client, "Halo 4 Artbook", category_string))
print(classify_item(client, "Too Many Losing Heroines!!! 7", category_string))
print(classify_item(client, "Spotify", category_string))
print(classify_item(client, "Google Cloud Bill", category_string))
print(classify_item(client, "Xbox Series S SSD", category_string))
print(classify_item(client, "Adobe Premiere Pro", category_string))
print(classify_item(client, "Creative Cloud", category_string))
print(classify_item(client, "Notebook", category_string))
print(classify_item(client, "1TB NVMe", category_string))

Gaming
Collectibles
Books & Literature
Digital Subscriptions
Digital Subscriptions
Gaming
Digital Subscriptions
Digital Subscriptions
Miscellaneous
Electronics & Accessories


In [9]:
for year, df in dfs.items():
    path = os.path.join("data", year + ".csv")
    df = classify_items_in_df(client, df, category_string, output_path=path)
    df.to_csv(path, index=False, encoding="utf-8")

Processing data\2024.csv: 100%|██████████| 216/216 [02:29<00:00,  1.44row/s]
